## Error handling is boring but this is the Exception

### Vito Minheere

Codebeez meetup 17th of May

Pygrunn 26th of May

Submitted at EuroPython 19th until 21st of July

# What is an Exception?

In [1]:
raise "Exception" 

TypeError: exceptions must derive from BaseException

An Exception is an object as many other things in Python. Exception extends the BaseException.

Extend it to create other Exceptions with the same attributes

Internal Python Exceptions should use BaseException, all others should use Exception



# Exception hierarchy

BaseException
 ├── GeneratorExit
 ├── KeyboardInterrupt
 ├── SystemExit
 └── Exception
      ├── ArithmeticError
      │    ├── FloatingPointError
      │    ├── OverflowError
      │    └── ZeroDivisionError
      ├── AttributeError
      ├── EOFError
      ├── LookupError
      │    ├── IndexError
      │    └── KeyError

Built-in errors also have their hierarchy. 

Exception hierarchy can serve an important purpose: 

the user does not have to know all the specific exceptions 

# Handling Exceptions

In [2]:
1 / 0

ZeroDivisionError: division by zero

# Handling Exceptions

In [3]:
try:
    1 / 0
    
except ZeroDivisionError as e:
    print("Zero Division raised")

Zero Division raised


# Handling Exceptions

In [4]:
try:
    1 / 0
    
except ArithmeticError as e:
#     print(type(e))
    print(isinstance(e, ArithmeticError))
    print(isinstance(e, ZeroDivisionError))
    print(isinstance(e, Exception))

    print("Arithmetic Error raised")
    raise e
    
except ZeroDivisionError as e:
    print(e)
    print("Zero Division raised")
    raise e

True
True
True
Arithmetic Error raised


ZeroDivisionError: division by zero

Arithmetic error is higher on the hierarchy than ZeroDivisionError

At most one except clause will trigger

The first except clause that matches will trigger


# Handling Exceptions

In [5]:
try:
    1 / 0

except Exception as e:
    print("Exception raised")
    
except ArithmeticError as e:
    print("Arithmetic Error raised")
    raise e
    
except ZeroDivisionError as e:
    print("Zero Division raised")
    raise e       


Exception raised


except Exception is even higher and will be executed first


# Handling Exceptions

In [6]:
try:
    thisssss_function_does_not_exist()
    
except Exception as e:
    pass
    
except ArithmeticError as e:
    print("Arithmetic Error raised")
    raise e
    
except ZeroDivisionError as e:
    print("Zero Division raised")
    raise e


Be aware that Exception will also catch all other Exceptions derived from it

Python is an interpreted language.
Python uses Exceptions for both reporting errors while running the code and interpreting code. 
If you catch all Exceptions it will also catch NameErrors during interpreting

# Handling Exceptions

In [50]:
import sys
try:
    1/0
except Exception as e: 
    print("SystemExit caught") 
    sys.exit()
finally:
    print("finally")


SystemExit caught
finally


AttributeError: 'tuple' object has no attribute 'tb_frame'

SystemExit is above Exception but extends BaseException

# Handling Exceptions

In [8]:
try:
    raise Exception('general exceptions not caught by specific handling')
except ValueError as e:
    print('we will not catch exception: Exception')



Exception: general exceptions not caught by specific handling

Hierarchy only travels up not down

# Handling Exceptions

In [9]:
try:
    1 / 0

except ArithmeticError as e:
    print("Error raised")
    
except ZeroDivisionError as e:
    print("Error raised")


Error raised


In [10]:
try:
    1 / 0

except (ArithmeticError, ZeroDivisionError) as e:
    print("Error raised")
    

Error raised


# NAASA, NASA As A Service


#### Launch a rocket via a Python module. 
#### Rocket will be supplied. However we have BYOC - Bring Your Own Crew

### No guarantees, only Exceptions

It is rocket science so a lot of things can go wrong. 

Before the launch you could exit your program and call it off if critical errors arise 

but you can’t just reset the rocket when it is in mid flight. 

In this case we are both the creator of the module and also the user.
We will use a bit of defensive programming to make sure edge cases will be handled and the rocket will not crash. 

## The Rocket

In [11]:
from classes.fuel_tank import FuelTank
from classes.command_centre import CommandCentre
from classes.engine import Engine

class Rocket:
    def __init__(self, name):
        self.name = name
        self.altitude = 0
        self.crew = []
        self.fuel_tank = FuelTank("Hydrogen", 100)
        self.command_centre = CommandCentre()
        self.engine = Engine(motors=4)
        self.startup_completed = False
        
    def __str__(self):
        return f"Rocket {self.name} has {self.engine.motors} motors and is at an altitude of {self.altitude} meters"
         
rocket = Rocket("Artemis 1")
print(rocket)

Rocket Artemis 1 has 4 motors and is at an altitude of 0 meters


In [12]:
class FuelTank:
    def __init__(self, fuel_type: str, max_volume: int) -> None:
        self.fuel_type = fuel_type
        self.volume = 0
        self.max_volume = max_volume

# Custom Exceptions

In [13]:
 class FuelTankError(Exception):
        pass

raise FuelTankError("Something is wrong with the fuel tank")

FuelTankError: Something is wrong with the fuel tank

To create a custom Exception you can create a class and extend Exception

Usually you can keep the base empty so that child Exception can set their own messages without them being overwritten by the base

# Custom Exceptions

In [16]:
class FuelTypeError(FuelTankError):
    
    def __init__(self, fuel_type, allowed_fuel):
        message = f"Invalid fuel type: {fuel_type}. Only '{allowed_fuel}' fuel is allowed."
        super().__init__(message)

class FuelLevelError(FuelTankError):  
    
    def __init__(self, max_volume, volume):
        amount_left_to_fill = max_volume - volume
        message = f"Max amount of fuel to add is {str(amount_left_to_fill)} liters" 
        self.value = amount_left_to_fill
        super().__init__(message)

# issubclass(FuelTypeError, FuelTankError)
# issubclass(FuelTypeError, Exception) 
raise FuelLevelError(10, 7)

FuelLevelError: Max amount of fuel to add is 3 liters

# Custom Exceptions

In [17]:
def refuel(self, fuel_type, amount):
    if fuel_type != self.fuel_type:
        raise FuelTypeError(fuel_type)
        
    elif (self.volume + amount) > self.max_volume:
        raise FuelLevelError(self.max_volume, self.volume)
        
    else:
        self.volume += amount
        print(f"Fuel tank refueled. Fuel level is now {self.volume} liter.")

Build a hierarchy in the custom exceptions. 

FuelException will be caught before FuelTankException when both are in the except clause.



# Custom Exceptions

In [18]:
try:
    rocket.fuel_tank.refuel("Diesel", 100)
    
except FuelTankError as e:
    print("FuelTankError")
    print(e)
    raise e
    
except FuelTypeError as e:
    print("FuelTypeError")
    print(e)

FuelTypeError: Invalid fuel type: Diesel. Only 'Hydrogen' fuel is allowed.

Due to the hierarchy the base FuelTankException can be caught first. 

When you reraise the Exception it shows the FuelTypeException.

End users of our module can put it as the last except block to catch any errors they might have missed.

# Custom Exceptions

In [19]:
try:
    rocket.fuel_tank.refuel("Diesel", 100)
    
except FuelLevelError as e:
    print("FuelLevelError")
    print(e)
    raise e
    
except FuelTankError as e:
    print("FuelTankError")
    print(e)
    raise

FuelTypeError: Invalid fuel type: Diesel. Only 'Hydrogen' fuel is allowed.

Here is an example of setting the custom "base" exception in the last block

The specific exception is missed in the code handling it

The hierarchy still manages to catch it and display the correct Exception happening

# Explicit chained exceptions

In [20]:
class Rocket:
    def __init__(self, name):
        self.name = name
        self.altitude = 0
        self.crew = []
        self.fuel_tank = FuelTank("Hydrogen", 100)
        self.command_centre = CommandCentre()
        self.engine = Engine(motors=4)
        self.startup_completed = False

As we saw earlier the Rocket class had an attribute called startup_completed. 
This could indicate a series of checks that have to be completed before being able to launch.

# Explicit chained exceptions

In [21]:
crew = ["Alice", "Bob"]
backup_crew = ["Carol"]

class RocketNotReadyError(Exception):
    pass


def personnel_check():
    try:
        print("\tThe captain's name is", crew[0])
        print("\tThe pilot's name is", crew[1])
        print("\tThe mechanic's name is", crew[2])
    except IndexError as e:
        raise RocketNotReadyError('Crew is incomplete') from e

print('Final check procedure')

personnel_check()

Final check procedure
	The captain's name is Alice
	The pilot's name is Bob


RocketNotReadyError: Crew is incomplete

An Exception can be caught and reraised again in the except clause

You can also use raise from to raise a new Exception but keep the old one in the stack trace

# Explicit chained exceptions

In [23]:
crew = ["Alice", "Bob"]
backup_crew = ["Carol"]

def personnel_check():
    try:
        print("\tThe captain's name is", crew[0])
        print("\tThe pilot's name is", crew[1])
        print("\tThe mechanic's name is", crew[2])
        print("\tThe engineer's name is", crew[3])

    except IndexError as e:
        print("Missing crew member, adding backup crew")
        
        try:
            crew.append(backup_crew.pop())
            personnel_check()
            
        except IndexError as e:
            raise RocketNotReadyError('Crew is incomplete') from e


# Explicit chained exceptions

In [24]:
try:
    print('Final check procedure')
    personnel_check()
except RocketNotReadyError as f:
    print('General exception: "{}", caused by "{}"'.format(f, f.__cause__))
    raise Exception("Can't launch Rocket today") from f

Final check procedure
	The captain's name is Alice
	The pilot's name is Bob
Missing crew member, adding backup crew
	The captain's name is Alice
	The pilot's name is Bob
	The mechanic's name is Carol
Missing crew member, adding backup crew
General exception: "Crew is incomplete", caused by "pop from empty list"


Exception: Can't launch Rocket today

The raise from can be extended again in another try except

The Exception will travel up the chain to where it wil be caught

See the traceback in the slide

# Suppressing errors

In [25]:
fuels = ["Diesel", "Hydrogen"]

fuels.remove("Gasoline")

ValueError: list.remove(x): x not in list

# Suppressing errors

In [26]:
fuels = ["Diesel", "Hydrogen"]

# (LBYL) Look before you leap:
def remove_from_list(a_list, element):
    if element in a_list:
        a_list.remove(element)
         
remove_from_list(fuels, "Gasoline")

# Suppressing errors

In [27]:
fuels = ["Diesel", "Hydrogen"]

# (EAFP) Easier to ask forgiveness than permission:
def remove_from_list(a_list, element):
    try:
        a_list.remove(element)
    except ValueError:
        pass

remove_from_list(fuels, "Gasoline")
print(fuels)

['Diesel', 'Hydrogen']


# Suppressing errors

In [28]:
fuels = ["Diesel", "Hydrogen"]

from contextlib import suppress

def remove_from_list(a_list, element):
    with suppress(ValueError):
        a_list.remove(element)
        
remove_from_list(fuels, "Gasoline")
print(fuels)

['Diesel', 'Hydrogen']


Easier to Ask for Forgiveness than Permission

As with any other mechanism that completely suppresses exceptions, this context manager should be used only to cover very specific errors where silently continuing with program execution is known to be the right thing to do.

# Suppressing errors

In [29]:
fuels = ["Diesel", "Hydrogen"]

def remove_from_list(a_list, element):
    try:
        a_list.remove(element)
    except ValueError:
        print(f"{element} not in list")

remove_from_list(fuels, "Gasoline")
print(fuels)

Gasoline not in list
['Diesel', 'Hydrogen']


# Suppressing errors

In [30]:
class CommandCentre:
    def __init__(self) -> None:
        self.signal = 100
        
    def generate_transmissions(self):
        messages = [
            "Message 1",
            {"number": 5},
            "Message 2",
            5,
            "Message 3",
        ]
        for message in messages:
            yield message  

The command centre will broadcast messages

Only the strings are interesting to us. The other will give Exceptions when we try to handle them

What could be a clean way to handle these?

# Suppressing errors

In [31]:
def receive_transmissions():
    centre = CommandCentre()
    transmission_generator = centre.generate_transmissions()
    while True:
        try:
            transmission = next(transmission_generator)
            
            try:
                transmission = transmission.strip()
            except AttributeError:
                continue
            except TypeError:
                continue
            else:
                print(f"Received: {transmission}")
                
        except StopIteration:
            print("End of transmission")
            break
        
receive_transmissions()

Received: Message 1
Received: Message 2
Received: Message 3
End of transmission


# Suppressing errors

In [32]:
from contextlib import suppress

def receive_transmissions():
    centre = CommandCentre()
    transmission_generator = centre.generate_transmissions()    
    while True:
        try:
            transmission = next(transmission_generator)
            
            with suppress((AttributeError, TypeError)):
                
                transmission = transmission.strip()
                print(f"Received: {transmission}")
                
        except StopIteration:
            print("End of transmission")
            break

receive_transmissions()

Received: Message 1
Received: Message 2
Received: Message 3
End of transmission


With suppress uses a ContextManager to handle the Exceptions. The result is the same as the code in the previous slide but it is easier to read and more Pythonic.

Return a context manager that suppresses any of the specified exceptions if they occur in the body of a with statement and then resumes execution with the first statement following the end of the with statement.

# Async Exception Handling

Async Python brings a whole load of other problems including multiple exceptions from different tasks

Python 3.11 brought new features including TaskGroups and ExceptionGroups

Both named Groups and released at the same time. They must be related.



In [33]:
issubclass(ExceptionGroup, Exception)

True

# Async Exception Handling

In [34]:
class MotorError(Exception):
    pass

class IgnitionError(MotorError):
    def __init__(self, motor) -> None:
        self.motor = motor
        message = f"Motor {motor} failed to ignite"
        super().__init__(message)

    def __str__(self):
        return f"Error in engine {self.motor}"

# Async Exception Handling

In [35]:
import asyncio
import random

async def fire_engine(motor):
    print(f"Attempting to fire motor {motor}")
    engine_fires_chance = random.random()
    
    if engine_fires_chance < 0.3:
        raise MotorError(f"Something went wrong in motor {motor}")
        
    elif engine_fires_chance >= 0.8:
        print(f"Motor {motor} failed to fire")
        raise IgnitionError(motor)
        
    print(f"Motor {motor} fired up!")

# Async Exception Handling

In [36]:
async def apply_throttle(motor):
    try:
        task = asyncio.create_task(fire_engine(motor))
        await task
        if task.done():
            ex = task.exception()
            if ex:
                print(ex)
    
    except IgnitionError as e:
        print(e)
        
#     except MotorError as e:
#         print(e)
        
await apply_throttle(1)
await apply_throttle(2)
await apply_throttle(3)
await apply_throttle(4)

Attempting to fire motor 1


MotorError: Something went wrong in motor 1

# Async Exception Handling

In [38]:
async def apply_throttle():
    results = await asyncio.gather(fire_engine(1), fire_engine(2), fire_engine(3), fire_engine(4))
    if results.done():
        ex = results.exception()
        if ex:
            print(ex)
            raise ex
                
await apply_throttle()

Attempting to fire motor 1
Motor 1 failed to fire
Attempting to fire motor 2
Motor 2 fired up!
Attempting to fire motor 3
Attempting to fire motor 4


IgnitionError: Error in engine 1

# Async Exception Handling

In [43]:
async def apply_throttle(motors):
    try:
        async with asyncio.TaskGroup() as tg:
            
            for x in range(motors):
                tg.create_task(
                    fire_engine(x)
                )

    except ExceptionGroup as e:
        print(e)
        print(e.exceptions) 
#     except* IgnitionError as e:
#         print(f"Handling IgnitionErrors: {e.exceptions}")
#         raise e.exceptions
        
#     except* MotorError as e:
#         print(f"Handling MotorErrors: {e.exceptions}")
#         raise e.exceptions

# Async Exception Handling

In [44]:

await apply_throttle(4) 

Attempting to fire motor 0
Attempting to fire motor 1
Attempting to fire motor 2
Motor 2 failed to fire
Attempting to fire motor 3
Motor 3 failed to fire
unhandled errors in a TaskGroup (4 sub-exceptions)
(MotorError('Something went wrong in motor 0'), MotorError('Something went wrong in motor 1'), IgnitionError('Motor 2 failed to ignite'), IgnitionError('Motor 3 failed to ignite'))


  + Exception Group Traceback (most recent call last):
  | ExceptionGroup:  (2 sub-exceptions)
  +-+---------------- 1 ----------------
    | Exception Group Traceback (most recent call last):
    |   File "/classes/engine.py", line 32, in apply_throttle
    |     async with asyncio.TaskGroup() as tg:
    |   File "/usr/lib/python3.11/asyncio/taskgroups.py", line 147, in __aexit__
    |     raise me from None
    | ExceptionGroup: unhandled errors in a TaskGroup (2 sub-exceptions)


   | ExceptionGroup: unhandled errors in a TaskGroup (2 sub-exceptions)
   +-+---------------- 1 ----------------
      | Traceback (most recent call last):
      |   File "/classes/engine.py", line 53, in fire_engine
      |     raise IgnitionError(motor)
      | classes.engine.IgnitionError: Error in engine 1
      +---------------- 2 ----------------
      | Traceback (most recent call last):
      |   File "/classes/engine.py", line 53, in fire_engine
      |     raise IgnitionError(motor)
      | classes.engine.IgnitionError: Error in engine 2

## So can we finally launch the Rocket?

In [46]:
from classes.rocket import Rocket, StartUpAbort, RocketNotReadyError
from classes.launchpad import Launchpad
from classes.engine import IgnitionError

rocket = Rocket("Artemis")
launchpad = Launchpad()

try:
    rocket.engine.ignite()
    rocket.startup_sequence()
except StartUpAbort as e:
    raise e


Motors ignited
Fuel tank refueled. Fuel level is now 100 liter.
Artemis refueled.
Communication works
All systems are go!
Startup sequence completed. Ready to launch
Finished startup check procedure


In [47]:
async def launch_rocket(rocket):
    try:
        launchpad.clear()
        await rocket.launch()
    except ExceptionGroup as e:
        print("Launch failed")
    else:
        print("\n")
        print(rocket)
    finally:
        launchpad.release()

await launch_rocket(rocket)

Clearing launchpad for launch
Applying throttle to the engine
Attempting to fire motor 0
Attempting to fire motor 1
Attempting to fire motor 2
Attempting to fire motor 3
Motor 0 failed to fire
Motor 2 fired up!
Motor 3 fired up!
Handling IgnitionErrors: (IgnitionError('Motor 0 failed to ignite'),)
Handling MotorErrors: (MotorError('Something went wrong in motor 1'),)
Atleast 2 engines fired, launching anyway
Artemis launched!


Rocket Artemis has 4 motors and is at an altitude of 100 meters
Releasing launchpad after launch sequence
